In [2]:
import cv2
import os
from utils.common_utils import read_json_data
from utils.eval_utils import is_bbox_overlap, top_bbox_from_scores
from sentence_transformers import SentenceTransformer, util
from PIL import Image

ModuleNotFoundError: No module named 'detectron2'

In [1]:
!export CUDA_VISIBLE_DEVICES=2

In [ ]:
# Data Directories
BASE_DIR = '/home/engine210/MMFinal/'
DATA_DIR = '/home/engine210/MMFinal/data/'
TARGET_DIR = "/home/engine210/MMFinal/viz/"


In [ ]:
test_samples = read_json_data(os.path.join(DATA_DIR, 'mmsys_anns', 'public_test_mmsys_final.json'))
v_data = test_samples[1]

In [ ]:
textual_sim = float(v_data['bert_base_score'])
textual_sim

In [ ]:
v_data

In [ ]:
img = Image.open(os.path.join(DATA_DIR, v_data['img_local_path']))

In [16]:
img.show()

In [19]:
stmodel = SentenceTransformer('clip-ViT-B-32')

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 31.75 GiB total capacity; 0 bytes already allocated; 5.50 MiB free; 0 bytes reserved in total by PyTorch)

In [6]:
bboxes = v_data['maskrcnn_bboxes']
top_bbox_c1 = top_bbox_from_scores(bboxes, score_c1)
top_bbox_c2 = top_bbox_from_scores(bboxes, score_c2)
print(top_bbox_c1, top_bbox_c2)

NameError: name 'score_c1' is not defined

In [15]:
score_c1, score_c2 = get_scores(v_data)

In [16]:
print(score_c1, score_c2)

tensor([-1.3500,  0.1246, -3.7564,  5.0216, -1.3501, -1.4075, -1.2110, -1.2906,
        -1.3245, -2.4942,  2.6325], device='cuda:0') tensor([-3.2680,  1.2437, -3.6021,  4.2952, -1.1264,  3.6721, -1.2736, -0.7548,
        -3.2077, -1.1968,  6.7783], device='cuda:0')


In [18]:
v_data['maskrcnn_bboxes']

[[389.6280517578125, 8.949727058410645, 609.6119995117188, 392.25555419921875],
 [890.519775390625, 439.1005554199219, 975.5325317382812, 547.5588989257812],
 [0.0, 453.0171813964844, 137.16851806640625, 547.4357299804688],
 [161.58926391601562, 431.6105651855469, 809.5007934570312, 542.5567016601562],
 [792.5897216796875, 479.9522705078125, 848.8662719726562, 546.4912719726562],
 [723.2586059570312, 434.73065185546875, 756.88818359375, 478.6017150878906],
 [234.8422088623047, 229.90127563476562, 717.4603881835938, 436.8536376953125],
 [180.153564453125, 437.98065185546875, 219.98043823242188, 488.01953125],
 [329.10504150390625, 230.89695739746094, 655.05126953125, 411.0627746582031],
 [64.88946533203125, 416.5194396972656, 172.52650451660156, 462.4348449707031]]

In [12]:
print([list(map(int, x)) for x in v_data['maskrcnn_bboxes']])

[[389, 72, 505, 373], [89, 312, 190, 396], [116, 225, 161, 288], [180, 225, 207, 271], [579, 193, 597, 249], [217, 225, 256, 267], [67, 237, 92, 275], [29, 240, 64, 276], [229, 298, 251, 330], [89, 205, 104, 228], [0, 0, 600, 400]]


In [6]:
def get_scores(v_data):
    """
        Computes score for the two captions associated with the image

        Args:
            v_data (dict): A dictionary holding metadata about on one data sample

        Returns:
            score_c1 (float): Score for the first caption associated with the image
            score_c2 (float): Score for the second caption associated with the image
    """
    checkpoint = torch.load(BASE_DIR + 'models/' + model_name + '.pt')
    combined_model.load_state_dict(checkpoint)
    combined_model.to(device)
    combined_model.eval()

    img_path = os.path.join(DATA_DIR, v_data["img_local_path"])
    bbox_list = v_data['maskrcnn_bboxes']
    bbox_classes = [-1] * len(bbox_list)
    img = cv2.imread(img_path)
    img_shape = img.shape[:2]
    bbox_list.append([0, 0, img_shape[1], img_shape[0]])  # For entire image (global context)
    bbox_classes.append(-1)
    cap1 = v_data['caption1_modified']
    cap2 = v_data['caption2_modified']

    img_tensor = [torch.tensor(img).to(device)]
    bboxes = [torch.tensor(bbox_list).to(device)]
    bbox_classes = [torch.tensor(bbox_classes).to(device)]

    if embed_type != 'use':
        # For Glove, Fasttext embeddings
        cap1_p = text_field.preprocess(cap1)
        cap2_p = text_field.preprocess(cap2)
        embed_c1 = torch.stack([text_field.vocab.vectors[text_field.vocab.stoi[x]] for x in cap1_p]).unsqueeze(
            0).to(device)
        embed_c2 = torch.stack([text_field.vocab.vectors[text_field.vocab.stoi[x]] for x in cap2_p]).unsqueeze(
            0).to(device)
    else:
        # For USE embeddings
        embed_c1 = torch.tensor(use_embed([cap1]).numpy()).to(device)
        embed_c2 = torch.tensor(use_embed([cap2]).numpy()).to(device)

    with torch.no_grad():
        z_img, z_t_c1, z_t_c2 = combined_model(img_tensor, embed_c1, embed_c2, 1, [embed_c1.shape[1]],
                                               [embed_c2.shape[1]], bboxes, bbox_classes)

    z_img = z_img.permute(1, 0, 2)
    z_text_c1 = z_t_c1.unsqueeze(2)
    z_text_c2 = z_t_c2.unsqueeze(2)

    # Compute Scores
    score_c1 = torch.bmm(z_img, z_text_c1).squeeze()
    score_c2 = torch.bmm(z_img, z_text_c2).squeeze()

    return score_c1, score_c2

In [ ]:
ours_correct = 0
lang_correct = 0

for i, v_data in enumerate(test_samples):
    actual_context = int(v_data['context_label'])
    
    language_context = 0 if float(v_data['bert_base_score']) >= textual_sim_threshold else 1

    pred_context = evaluate_context_with_bbox_overlap(v_data)

    if pred_context == actual_context:
        ours_correct += 1

    if language_context == actual_context:
        lang_correct += 1

print("Cosmos Accuracy", ours_correct / len(test_samples))
print("Language Baseline Accuracy", lang_correct / len(test_samples))